&emsp;&emsp;在心理学框架中，人类根据随意线索和不随意线索选择注意点。比如在一堆黑白的物件中，有一个红色的咖啡杯，那么这个**红色的咖啡杯就是不随意线索**。

&emsp;&emsp;卷积、全连接、池化层都只考虑**不随意线索**。比如像池化层将里面最大的那个像素提取出来。

&emsp;&emsp;注意力机制则显示的考虑随意线索：

1. 随意线索被称之为查询(`query`), 比如像想要去做某件事情。

2. 每个输入是一个值(`value`)和不随意线索(`key`)的对。`key`可以看作是物品本身所带的属性，`value`是我对这件物品的价值判定。

3. 通过注意力池化层来有偏向性地选择某些输入, 就是选择某一些`key`，`value`对。

## 非参数注意力池化层

&emsp;&emsp;给定数据$(x_{i}, y_{i}), i = 1, \cdots, n$, $x$可以看作是`key`，$y$可以看作是`value`。


1. 平均池化是最简单的方案: $f(x) = \frac{1}{n} \sum_{i} y_{i}$。也就是给定查询`query`, 也就是$f(x)$中的$x$, 直接返回所有候选项`value`的均值, 不管输入查询$x$。

2. 好一点的办法是`Nadaraya-Watson`核回归:

$$
f(x)=\sum_{i=1}^{n} \frac{K\left(x-x_{i}\right)}{\sum_{j=1}^{n} K\left(x-x_{j}\right)} y_{i}
$$

&emsp;&emsp;对于给定的`query`, 也就是$f(x)$中的$x$，去与所有的候选的`key`相减，再做一个$K$函数的映射。再除以总和，就可以得到一个相对重要性。之后用这个东西加权对$y_{i}$求和。

&emsp;&emsp;直观理解就是来了一个新的数据$x$，将其与其接近的一些数据拿出来看看就可以了，其余的就不管了。


&emsp;&emsp;如果$K$选的是一个高斯核$K(\mu) = \frac{1}{\sqrt{2\pi}} exp(-\frac{u^{2}}{2})$

&emsp;&emsp;那么:

$$
\begin{aligned}
f(x) &=\sum_{i=1}^{n} \frac{\exp \left(-\frac{1}{2}\left(x-x_{i}\right)^{2}\right)}{\sum_{j=1}^{n} \exp \left(-\frac{1}{2}\left(x-x_{j}\right)^{2}\right)} y_{i} \\
&=\sum_{i=1}^{n} \operatorname{softmax}\left(-\frac{1}{2}\left(x-x_{i}\right)^{2}\right) y_{i}
\end{aligned}
$$

## 参数注意力池化层

&emsp;&emsp;在之前的基础上，引入可以学习的$w$:

$$
f(x)=\sum_{i=1}^{n} \operatorname{softmax}\left(-\frac{1}{2}\left(\left(x-x_{i}\right) w\right)^{2}\right) y_{i}
$$

- **小节**

&emsp;&emsp;心理学认为人通过随意线索和不随意线索选择注意点。

&emsp;&emsp;注意力机制中，通过`query`(随意线索)和`key`(不随意线索)来有偏向性地选择输入。可以一般的写作$f(x)=\sum_{i} \alpha\left(x, x_{i}\right) y_{i}$，这里$\alpha\left(x, x_{i}\right)$是注意力权重。

## 注意力分数

&emsp;&emsp;我们回顾一下:

$$
f(x)=\sum_{i} \alpha\left(x, x_{i}\right) y_{i}=\sum_{i=1}^{n} \operatorname{softmax}\left(-\frac{1}{2}\left(x-x_{i}\right)^{2}\right) y_{i}
$$

&emsp;&emsp;其中$\alpha$是注意力权重(`attention weights`)，它是由注意力分数(`attention scoring`)经过`softmax`之后得到的。

### 扩展到高维度

&emsp;&emsp;假设`query` $\mathbf{q} \in \mathbb{R}^{q}$, 有$m$对`key-value` ($\mathbf{k}_{1}$, $\mathbf{v}_{1}$),$\cdots,$。$\mathbf{k}_{i} \in \mathbb{R}^{k}$, $\mathbf{v}_{i} \in \mathbb{R}^{v}$, 他们的维度都可以不一样。

&emsp;&emsp;注意力池化层:

$$
f\left(\mathbf{q},\left(\mathbf{k}_{1}, \mathbf{v}_{1}\right), \ldots,\left(\mathbf{k}_{m}, \mathbf{v}_{m}\right)\right)=\sum_{i=1}^{m} \alpha\left(\mathbf{q}, \mathbf{k}_{i}\right) \mathbf{v}_{i} \in \mathbb{R}^{v}
$$

&emsp;&emsp;其中查询$\mathbf{q}$和键$\mathbf{k}_{i}$的注意力权重(标量)是通过注意力评分函数$a$将两个向量映射成标量，再经过`softmax`运算得到的:

&emsp;&emsp;$\alpha$的设计通常可以写成如下形式:

$$
\alpha\left(\mathbf{q}, \mathbf{k}_{i}\right)=\operatorname{softmax}\left(a\left(\mathbf{q}, \mathbf{k}_{i}\right)\right)=\frac{\exp \left(a\left(\mathbf{q}, \mathbf{k}_{i}\right)\right)}{\sum_{j=1}^{m} \exp \left(a\left(\mathbf{q}, \mathbf{k}_{j}\right)\right)} \in \mathbb{R}
$$


&emsp;&emsp;正如我们所看到的，选择不同的注意力评分函数$a$会导致不同的注意力汇聚操作。


### Additive Attention

&emsp;&emsp;一般来说，当查询和键是不同长度的矢量时，可以使用加性注意力作为评分函数。给定查询$\mathbf{q} \in \mathbb{R}^{q}$和$\mathbf{k} \in \mathbb{R}^{k}$, 加性注意力(`additive attention`)的评分函数为:


$$
a(\mathbf{q}, \mathbf{k})=\mathbf{w}_{v}^{\top} \tanh \left(\mathbf{W}_{q} \mathbf{q}+\mathbf{W}_{k} \mathbf{k}\right) \in \mathbb{R}
$$

&emsp;&emsp;其中可学习的参数有: $\mathbf{W}_{k} \in \mathbb{R}^{h \times k}, \mathbf{W}_{q} \in \mathbb{R}^{h \times q}, \mathbf{v} \in \mathbb{R}^{h}$。$\mathbf{W}_{k}$相当于将一个纬度为$k$的`key`映射到$h$, $\mathbf{W}_{q}$相当于将一个纬度为$q$的`query`映射到$h$。等价于将`query`和`key`合并起来放入到一个隐藏大小为$h$输出大小为`1`的单隐藏层`MLP`中。

### Scaled Dot-Product Attention

&emsp;&emsp;如果`query`和`key`都是同样长度的$\mathbf{q}, \mathbf{k}_{i} \in \mathbb{R}^{d}$, 那么可以:

$$
a\left(\mathbf{q}, \mathbf{k}_{\mathbf{i}}\right)=\left\langle\mathbf{q}, \mathbf{k}_{\mathbf{i}}\right\rangle / \sqrt{d}
$$

&emsp;&emsp;除以根号$\sqrt{d}$的话，可以使得其对参数不那么敏感。

- **向量化版本**

1. 向量化版本可以写成: $\mathbf{Q} \in \mathbb{R}^{n \times d}, \mathbf{K} \in \mathbb{R}^{m \times d}, \mathbf{V} \in \mathbb{R}^{m \times v}$。

2. 注意力分数可以表示为:$a(\mathbf{Q}, \mathbf{K})=\mathbf{Q} \mathbf{K}^{T} / \sqrt{d} \in \mathbb{R}^{n \times m}
$

3. 注意力池化: $f=\operatorname{softmax}(a(\mathbf{Q}, \mathbf{K})) \mathbf{V} \in \mathbb{R}^{n \times v}$。

## Attention机制

<img src="../images/scaled-dot-product-attention.png" width="50%">


> - 图中`Decoder`的隐藏状态向量`query` $q_t$ 与`Encoder`的隐藏状态向量序列`key` $K=[k_1, k_2, k_3, k_4]$ 每个向量做点积，每个 `key`向量对应一个分数。
- 然后`sofmax`转换为权重分布，权重值越大可以认为越“关注”该处向量的信息。
- 每个位置的 $v$ 与该处的权重相乘，然后求和得到 $q_t$ 对 $K$ 做注意力的输出向量。

## Pytorch实现

In [13]:
import torch, math, torch.nn.functional as F


def attention(query, key, value, mask=None, dropout=None):
    """
    query : batch, target_len, feats
    key   : batch, seq_len,    feats
    value : batch, seq_len,    val_feats
    
    return: batch, target_len, val_feats
    """
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim=-1)

    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

`query`与`key`去做点积，能够得到`seq_len`个点积之后的结果。再做个`mask`和`softmax`之后，我们就可以得到`attention`，此时的`attention`的长度是$target\_len \times seq\_len$, 再与编码器的value输出纬度$seq\_len \times val\_feats$做矩阵乘法之后，就可以得到解码器`decoder`的输出：

In [14]:
def test_attention():
    query = torch.randn(3, 5, 4)  # batch, target_len, feats
    key = torch.randn(3, 6, 4)  # batch, seq_len, feats
    value = torch.randn(3, 6, 8)  # batch, seq_len, val_feats
    attn, _ = attention(query, key, value)
    print(attn.shape)
    assert attn.shape == (3, 5, 8)
    print("Test passed")

test_attention()

torch.Size([3, 5, 8])
Test passed


## 位置编码

- 如果模型的输出会随着输入文本数据顺序的变化而变化，那么这个模型就是关于位置敏感的，反之则是位置不敏感的。设模型为函数$y=f(x)$，其中输入为一个词序列$x=\{x_1,x_2...x_n\}$。将$x$任意重排序$\hat{x}=\text{shuffle}(x)$，都有$f(\hat{x})=f(x)$，则模型是关于位置不敏感的。
    - 当使用对位置不敏感(position-insensitive)的模型对文本数据建模的时候，才需要额外使用positional encoding。
   
   
- RNN和textCNN都是关于位置敏感的，使用它们对文本数据建模时，模型结构天然考虑了文本中词与词之间的顺序关系。   
- 以attention为核心的transformer则是位置不敏感的，使用这一类位置不敏感的模型的时候需要额外加入positional encoding引入文本中词与词的顺序关系。